In [44]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from albumentations import Compose, Blur
from albumentations.pytorch import ToTensorV2
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import albumentations as A
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import f1_score, recall_score, precision_score
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
!pip install wandb

In [3]:
import wandb

In [4]:
df = pd.read_csv('/kaggle/input/ham1000-segmentation-and-classification/GroundTruth.csv')
df

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
random_state = 4747

In [6]:
df.describe()

,MEL,NV,BCC,AKIEC,BKL,DF,VASC
count,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000
mean,0.111133,0.669496,0.051323,0.032651,0.109735,0.011483,0.014179
std,0.314313,0.470418,0.220667,0.177730,0.312575,0.106546,0.118233
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image       10015 non-null  object 
 1   MEL         10015 non-null  float64
 2   NV          10015 non-null  float64
 3   BCC         10015 non-null  float64
 4   AKIEC       10015 non-null  float64
 5   BKL         10015 non-null  float64
 6   DF          10015 non-null  float64
 7   VASC        10015 non-null  float64
 8   Sum of row  10015 non-null  float64
dtypes: float64(8), object(1)
memory usage: 704.3+ KB


In [8]:
df.columns

Index(['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], dtype='object')

* Actinic keratoses and intraepithelial carcinoma / Bowen's disease (AKIEC),
* basal cell carcinoma (BCC),
*  benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, BKL),
*  dermatofibroma (DF),
*  melanoma (MEL),
*  melanocytic nevi (NV)
*  vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, VASC).

1. Actinic Keratoses and Intraepithelial Carcinoma / Bowen's Disease (AKIEC)
Актинічний кератоз (AK): Це передраковий стан, спричинений хронічним впливом ультрафіолетового випромінювання (УФ). Виявляється у вигляді шорстких, лускатих ділянок шкіри, що зазвичай виникають на відкритих частинах тіла, таких як обличчя, шия, руки. AK може трансформуватися у плоскоклітинний рак шкіри.
Хвороба Боуена (in situ плоскоклітинний рак): Це рання форма раку шкіри, яка локалізується у верхньому шарі шкіри (епідермісі). Вона може згодом стати інвазивним плоскоклітинним раком, якщо не лікувати.
2. Basal Cell Carcinoma (BCC)
Базальноклітинна карцинома: Це найпоширеніший вид раку шкіри, але водночас найменш агресивний. BCC зазвичай виникає через довготривалий вплив УФ-променів і розвивається дуже повільно. Може мати вигляд перлиноподібних вузликів або виразок, які не загоюються. BCC рідко метастазує, але може спричинити локальне пошкодження тканин.
3. Benign Keratosis-like Lesions (BKL)
Доброякісні кератозоподібні ураження: Це неракові утворення, що включають такі стани, як:
Сонячні лентиго (solar lentigines): Пігментні плями, які з’являються через вплив сонця. Часто виникають у людей старшого віку.
Себорейний кератоз (seborrheic keratosis): Доброякісні ураження шкіри, які зазвичай мають вигляд бородавок або плям, що можуть бути пігментованими.
Кератози ліхеноподібні (lichen-planus-like keratosis): Це окремий вид уражень, які мають певні схожості з актиничними кератозами.
4. Dermatofibroma (DF)
Дерматофіброма: Це доброякісне ураження шкіри, що зазвичай виглядає як маленька, тверда вузликова структура. Вони не є небезпечними і рідко потребують лікування, але можуть бути видалені з естетичних причин або у випадку дискомфорту.
5. Melanoma (MEL)
Меланома: Один із найбільш небезпечних видів раку шкіри, який виникає з пігментних клітин — меланоцитів. Меланома може розвиватися на шкірі або на слизових оболонках. Це агресивний вид раку, який може швидко метастазувати до інших органів. Раннє виявлення значно покращує прогноз лікування.
6. Melanocytic Nevi (NV)
Меланоцитарні невуси (родимки): Це скупчення меланоцитів у певних ділянках шкіри. Більшість невусів є доброякісними і не вимагають лікування, але деякі можуть мати потенціал перетворитися на меланому, особливо якщо є ознаки зміни їхньої форми, кольору або розміру.
7. Vascular Lesions (VASC)
Судинні ураження: Це група уражень, пов’язаних з аномаліями судин, до яких відносяться:
Ангіоми: Доброякісні пухлини, які розвиваються з кровоносних судин.
Ангіокератоми: Доброякісні судинні утворення, які зазвичай виникають на шкірі.
Піогенні гранульоми: Вони можуть розвинутися після травм і виглядають як червоні або рожеві вузлики, часто кровоточать.
Геморагії: Ураження, пов’язані з крововиливом в шкіру або під неї.
Кожен із цих станів вимагає ретельної діагностики, особливо для диференціації доброякісних утворень від злоякісних.

In [9]:
for col in ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']:
    print(f'{col}: {df[col].sum()}')

MEL: 1113.0
NV: 6705.0
BCC: 514.0
AKIEC: 327.0
BKL: 1099.0
DF: 115.0
VASC: 142.0


In [11]:
df['Sum of row'] = df[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].sum(axis=1)

In [11]:
# Тобто у кожному рядку як мінімум одна одиниця, і максимум одна
df.describe()

,MEL,NV,BCC,AKIEC,BKL,DF,VASC,Sum of row
count,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000,10015.000000,10015.0
mean,0.111133,0.669496,0.051323,0.032651,0.109735,0.011483,0.014179,1.0
std,0.314313,0.470418,0.220667,0.177730,0.312575,0.106546,0.118233,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [12]:
image_folder = '/kaggle/input/ham1000-segmentation-and-classification/images/'

# Отримуємо список усіх файлів у папці
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

# Створюємо змінну для збереження розміру першого зображення
first_image_size = None
same_size = True

# Перевіряємо розмір кожного зображення
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    with Image.open(image_path) as img:
        if first_image_size is None:
            first_image_size = img.size  # Зберігаємо розмір першого зображення
        else:
            if img.size != first_image_size:
                same_size = False
                print(f"Зображення {image_file} має інший розмір: {img.size}")
                break

if same_size:
    print("Усі зображення мають однаковий розмір:", first_image_size)
else:
    print("Деякі зображення мають різні розміри.")

Усі зображення мають однаковий розмір: (600, 450)


# Cтворення датасету

Загалом, можна взяти 7-10% з усієї вибірки. У нас 7 можливих лейблів. Візьмемо по 100 на лейбл.

In [100]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

def create_balanced_datasets(df, total_photos, val_size, test_size, random_state=4747):
    labels = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
    # Створюємо колонку "target" для стратифікації
    df['target'] = df[labels].idxmax(axis=1)

    # Визначаємо кількість зображень для кожного класу пропорційно їхньому розміру
    total_available_photos = df.groupby('target').size()
    max_photos_per_class = total_photos // len(labels)

    # Створюємо збалансований датасет без oversampling
    balanced_df = pd.DataFrame()
    for label in labels:
        available_photos = total_available_photos[label]
        num_photos_to_take = min(available_photos, max_photos_per_class)
        label_df = df[df['target'] == label].sample(n=num_photos_to_take, random_state=random_state)
        balanced_df = pd.concat([balanced_df, label_df], axis=0)

    # Тепер здійснюємо oversampling для тренувального датасету
    train_df, temp_df = train_test_split(
        balanced_df, 
        test_size=val_size + test_size, 
        random_state=random_state, 
        stratify=balanced_df['target']
    )

    for label in labels:
        label_df = train_df[train_df['target'] == label]
        current_class_size = len(label_df)
        # print(current_class_size)
        # print(max_photos_per_class)
        if current_class_size < max_photos_per_class:
            # Oversample класи з меншим числом зображень
            label_df_oversampled = resample(
                label_df, 
                replace=True, 
                n_samples=max_photos_per_class - current_class_size, 
                random_state=random_state
            )
            train_balanced_df = pd.concat([train_df, label_df_oversampled], axis=0)
        else:
            train_balanced_df = pd.concat([train_balanced_df, label_df], axis=0)

    # Тепер ми можемо розділити залишкові дані на валідацію та тест без змішування
    val_df, test_df = train_test_split(
        temp_df, 
        test_size=test_size / (val_size + test_size), 
        random_state=random_state, 
        stratify=temp_df['target']
    )

    print(f"Train set size: {len(train_balanced_df)}")
    print(f"Validation set size: {len(val_df)}")
    print(f"Test set size: {len(test_df)}")

    return train_balanced_df, val_df, test_df


In [101]:
train_df, val_df, test_df = create_balanced_datasets(df, total_photos=2000, val_size=0.2, test_size=0.1)

Train set size: 1363
Validation set size: 336
Test set size: 169


In [87]:
train_df

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,target,Sum of row
1784,ISIC_0026090,0.0,0.0,1.0,0.0,0.0,0.0,0.0,BCC,1.0
8754,ISIC_0033060,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL,1.0
7779,ISIC_0032085,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV,1.0
1209,ISIC_0025515,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL,1.0
3585,ISIC_0027891,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV,1.0
...,...,...,...,...,...,...,...,...,...,...
2963,ISIC_0027269,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC,1.0
8786,ISIC_0033092,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC,1.0
8626,ISIC_0032932,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC,1.0
4793,ISIC_0029099,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC,1.0


In [102]:
train_df = train_df.drop(['Sum of row'], axis=1)
val_df = val_df.drop(['Sum of row'], axis=1)
test_df = test_df.drop(['Sum of row'], axis=1)

In [89]:
train_df

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,target
1784,ISIC_0026090,0.0,0.0,1.0,0.0,0.0,0.0,0.0,BCC
8754,ISIC_0033060,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL
7779,ISIC_0032085,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
1209,ISIC_0025515,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL
3585,ISIC_0027891,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
...,...,...,...,...,...,...,...,...,...
2963,ISIC_0027269,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC
8786,ISIC_0033092,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC
8626,ISIC_0032932,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC
4793,ISIC_0029099,0.0,0.0,0.0,0.0,0.0,0.0,1.0,VASC


In [103]:
class SkinLesionClassificationDataset(Dataset):
    def __init__(self, csv_file, image_dir, mask_dir, transform=None):
        """
        Датасет для класифікації родимок (вирізання під маску).
        
        csv_file: шлях до CSV-файлу з інформацією про зображення та мітки класів.
        image_dir: директорія з зображеннями.
        mask_dir: директорія з масками (сегментаціями).
        transform: трансформації, які застосовуються до зображень.
        """
        self.annotations = csv_file
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.annotations = self.annotations.reset_index(drop=True)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Зчитуємо назву фото та маски
        img_name = os.path.join(self.image_dir, self.annotations.loc[idx, 'image'] + '.jpg')
        mask_name = os.path.join(self.mask_dir, self.annotations.loc[idx, 'image'] + '_segmentation.png')
        
        # Завантажуємо зображення та маску
        image = cv2.imread(img_name)
        mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
        
        if image is None or mask is None:
            raise FileNotFoundError(f"Не знайдено файл: {img_name} або {mask_name}")
    
        # Використовуємо маску, щоб вирізати частину зображення
        masked_image = self.apply_mask(image, mask)
        
        # Зчитуємо мітку класу (в один гарячий вектор) та перетворюємо в індекс класу
        label = self.annotations.loc[idx, ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values
        label = label.argmax()  # Знаходимо мітку з найвищим значенням
    
        # Застосовуємо трансформації, якщо вони є
        if self.transform:
            # Застосування трансформацій до зображення
            transformed = self.transform(image=masked_image)
            masked_image = transformed['image'] 
    
        return masked_image, label


    def get_summary(self):
        """
        Повертає підсумок датасету.
        """
        summary = {
            "Total Samples": len(self),
            "Class Distribution": self.annotations[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].sum().to_dict()
        }
        return summary

    def get_sample(self, idx):
        """
        Повертає зображення та мітку за індексом.
        """
        return self.__getitem__(idx)
    
    def apply_mask(self, image, mask):
        """
        Вирізає область зображення, яка відповідає масці.
        
        image: зображення родимки
        mask: сегментаційна маска
        
        Повертає вирізану область.
        """
        # Маска має бути бінарною (чорне-біле)
        mask = (mask > 0).astype(np.uint8)
        
        # Використовуємо маску, щоб вирізати лише частину зображення
        masked_image = cv2.bitwise_and(image, image, mask=mask)
        
        # Визначаємо область, де маска не порожня
        coords = cv2.findNonZero(mask)
        x, y, w, h = cv2.boundingRect(coords)  # Знаходимо обмежуючий прямокутник
        
        # Вирізаємо область з зображення
        cropped_image = masked_image[y:y+h, x:x+w]
        
        return cropped_image

In [133]:
transform_classification = A.Compose([
    A.Resize(height=600, width=450, p=1.0),  # Зміна розміру зображення до 600 на 450
    A.Blur(blur_limit=3, p=0.5),  # Додаємо блюр з ймовірністю 50%
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT),  # Зміщення, масштабування та обертання
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Нормалізація для ResNet
    ToTensorV2()  # Перетворення в тензор PyTorch
])

In [134]:
mask_dir =  '/kaggle/input/ham1000-segmentation-and-classification/masks/'
image_dir = '/kaggle/input/ham1000-segmentation-and-classification/images/'

In [135]:
train_df_dataset = SkinLesionClassificationDataset(train_df, image_dir, mask_dir, transform=transform_classification)
val_df_dataset = SkinLesionClassificationDataset(val_df, image_dir, mask_dir, transform=transform_classification)
test_df_dataset = SkinLesionClassificationDataset(test_df, image_dir, mask_dir, transform=transform_classification)

In [107]:
train_df_dataset[800]

(tensor([[[ 0.2111,  0.1939,  0.1768,  ..., -2.1179, -2.1179, -2.1179],
          [ 0.2111,  0.1939,  0.1597,  ..., -2.1179, -2.1179, -2.1179],
          [ 0.2111,  0.1939,  0.1597,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[ 0.1001,  0.0826,  0.0651,  ..., -2.0357, -2.0357, -2.0357],
          [ 0.1001,  0.0826,  0.0651,  ..., -2.0357, -2.0357, -2.0357],
          [ 0.1001,  0.0826,  0.0651,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[ 1.0714,  1.0539,  1.0539,  ..., -1.8044, -1.8044, -1.8044],
          [ 1.0714,  1.0539,

In [108]:
train_summary = train_df_dataset.get_summary()
print(train_summary)

{'Total Samples': 1363, 'Class Distribution': {'MEL': 200.0, 'NV': 199.0, 'BCC': 199.0, 'AKIEC': 200.0, 'BKL': 199.0, 'DF': 81.0, 'VASC': 285.0}}


In [109]:
print(val_df_dataset.get_summary())

{'Total Samples': 336, 'Class Distribution': {'MEL': 57.0, 'NV': 57.0, 'BCC': 57.0, 'AKIEC': 56.0, 'BKL': 57.0, 'DF': 23.0, 'VASC': 29.0}}


In [136]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['target']), y=train_df['target'])
class_weights_dict = dict(enumerate(class_weights))
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [137]:
val_summary = val_df_dataset.get_summary()
print(val_summary)

{'Total Samples': 336, 'Class Distribution': {'MEL': 57.0, 'NV': 57.0, 'BCC': 57.0, 'AKIEC': 56.0, 'BKL': 57.0, 'DF': 23.0, 'VASC': 29.0}}


In [112]:
test_summary = test_df_dataset.get_summary()
print(test_summary)

{'Total Samples': 169, 'Class Distribution': {'MEL': 28.0, 'NV': 29.0, 'BCC': 29.0, 'AKIEC': 29.0, 'BKL': 29.0, 'DF': 11.0, 'VASC': 14.0}}


In [113]:
print(train_df_dataset.annotations.index)

RangeIndex(start=0, stop=1363, step=1)


In [37]:
def display_sample(dataset, index):
    """
    Виводить зображення та його мітку з заданого датасету.

    Parameters:
        dataset: Об'єкт датасету, який має метод get_sample.
        index: Індекс зразка, який потрібно вивести.
    """
    # Отримання зображення та мітки
    image, label = dataset.get_sample(index)

    # Виведення зображення
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Конвертуємо BGR в RGB для коректного відображення
    plt.title(f"Label: {label}")
    plt.axis('off')  # Сховати осі
    plt.show()

In [38]:
display_sample(train_df_dataset, 38)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
display_sample(train_df_dataset, 87)

# МОДЕЛІ

In [162]:
!wandb login 492953ddcda0576b6e6ebf89860aed0ccd177efe

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [165]:
run_name = f"resnet50"

In [167]:
import wandb
wandb.login()
print(wandb.run.status)  # Дізнатися статус підключення до сервісу


AttributeError: 'NoneType' object has no attribute 'status'

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
wandb.init(sync_tensorboard=False)  # Вимкнути синхронізацію з TensorBoard


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [166]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Skin cancer detection",

    # track hyperparameters and run metadata
    config={
    "epochs" : 50,
    "batch_size" : 8,
    "learning_rate" : 0.001,
    # "model_architecture": "ResNet50", 
    # "optimizer": "Adam",
    # "loss_function": "CrossEntropy",
    },
    name=run_name
)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


KeyboardInterrupt: 

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [142]:
class SkinLesionClassifier_Resnet50(nn.Module):
    def __init__(self, num_classes=7, freeze_base=True):
        super(SkinLesionClassifier_Resnet50, self).__init__()
        # Використовуємо переднавчену модель ResNet50
        self.base_model = models.resnet50(pretrained=True)
        # Заморожуємо ваги базової моделі (крім останнього блоку)
        if freeze_base:
            for param in self.base_model.parameters():
                param.requires_grad = False
            for param in self.base_model.layer4.parameters():
                param.requires_grad = True
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)  # Без Dropout
        )
         
    def forward(self, x):
        return self.base_model(x)

In [158]:
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)


    return f1, recall, precision

In [151]:
config = wandb.config

In [152]:
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

In [153]:
# Ініціалізація моделі, втрат та оптимізатора
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SkinLesionClassifier_Resnet50(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [154]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0, verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=5, verbose=True)

In [155]:
# Створення DataLoader для навчального, валідаційного та тестового наборів
train_loader = DataLoader(train_df_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_df_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_df_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

In [159]:
# Цикл навчання
for epoch in range(config.epochs):
    model.train()  # Переводимо модель в режим навчання
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Обнуленн я градієнтів
        optimizer.zero_grad()

        # Прямий прохід
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Зворотній прохід та оптимізація
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Зберігаємо передбачення для валідації
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Витягти середнє значення втрат
    avg_loss = running_loss / len(train_loader)

    # Обчислення метрик
    f1, recall, precision = calculate_metrics(all_labels, all_preds)

    # Логування в WandB
    wandb.log({"epoch": epoch + 1, "loss": avg_loss, "f1_score": f1, "recall": recall, "precision": precision})

    print(f'Epoch [{epoch+1}/{config.epochs}], Loss: {avg_loss:.4f}, F1-score: {f1:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}')

    # Валідація
    model.eval()  # Переводимо модель в режим валідації
    val_running_loss = 0.0
    val_preds = []
    val_labels_list = []  # Основний список для збереження міток
    
    with torch.no_grad():  # Не рахуємо градієнти під час валідації
        for val_images, batch_labels in val_loader:  # Використовуємо batch_labels замість val_labels
            val_images, batch_labels = val_images.to(device), batch_labels.to(device)
    
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, batch_labels)
    
            val_running_loss += val_loss.item()
    
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_preds.extend(val_predicted.cpu().numpy())
            val_labels_list.extend(batch_labels.cpu().numpy())  # Зберігаємо мітки
    
    # Витягти середнє значення втрат для валідації
    avg_val_loss = val_running_loss / len(val_loader)
    
    # Обчислення метрик для валідації
    val_f1, val_recall, val_precision = calculate_metrics(val_labels_list, val_preds)
    
    # Логування в WandB
    wandb.log({
        "val_loss": avg_val_loss,
        "val_f1_score": val_f1,
        "val_recall": val_recall,
        "val_precision": val_precision
    })
    
    print(f'Validation Loss: {avg_val_loss:.4f}, '
          f'Validation F1-score: {val_f1:.4f}, '
          f'Validation Recall: {val_recall:.4f}, '
          f'Validation Precision: {val_precision:.4f}')

    # Learning Rate Scheduler
    scheduler.step(avg_val_loss)

    # Early Stopping
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

Epoch [1/50], Loss: 1.2169, F1-score: 0.4400, Recall: 0.4711, Precision: 0.5143
Validation Loss: 1.1696, Validation F1-score: 0.4681, Validation Recall: 0.5332, Validation Precision: 0.6324
Epoch [2/50], Loss: 1.0780, F1-score: 0.5278, Recall: 0.5409, Precision: 0.5819
Validation Loss: 1.0505, Validation F1-score: 0.5150, Validation Recall: 0.5412, Validation Precision: 0.6246
Epoch [3/50], Loss: 0.9738, F1-score: 0.5888, Recall: 0.5914, Precision: 0.6115
Validation Loss: 0.9284, Validation F1-score: 0.6156, Validation Recall: 0.6225, Validation Precision: 0.6860
Epoch [4/50], Loss: 0.8619, F1-score: 0.6315, Recall: 0.6296, Precision: 0.6602
Validation Loss: 0.9389, Validation F1-score: 0.6124, Validation Recall: 0.6150, Validation Precision: 0.7169
EarlyStopping counter: 1 out of 5
Epoch [5/50], Loss: 0.8123, F1-score: 0.6566, Recall: 0.6555, Precision: 0.6724
Validation Loss: 0.9022, Validation F1-score: 0.6308, Validation Recall: 0.6348, Validation Precision: 0.6729
Epoch [6/50], Lo

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
